# Make GWAS blocks

This notebooks partitions GWAS signal into blocks and extract the smallest one per block.

Using SQL via `pandas` -- here I do it via `pandas` because I believe my computer has enough memory (64GB) for the job. But in practice one can build the input data to a database and adapt the SQL queries to running on physical databases.

The parameter `--get-top` will instead only keep the top variant per block.

## Usage

```
sos run GWAS_blocks --filename ~/Downloads/combined_pm_MVP.rds --increase-by 500000 --cwd ~/tmp/18-Jul-2020
```

## Data

The input data I received is in RDS format and looks like:

```
> head(readRDS('combined_lfsr_MVP.rds'))
                  hdl       ldl        tg        tc
1:100000012 0.1787311 0.4427086 0.2006725 0.5006527
1:10000006  0.6073020 0.5758349 0.6109343 0.5880965
1:100000507 0.4475880 0.3671566 0.4273243 0.3784221
1:100000827 0.1137560 0.4597413 0.1258910 0.5036999
1:100000843 0.4532421 0.5815971 0.4456185 0.5834053
1:100000948 0.3151570 0.5017485 0.3500289 0.4714799
```

In [2]:
[global]
parameter: cwd = path('.')
# File input is zipped and is chr:pos score_1, ...
parameter: filename = path('~/Downloads/combined_pm_MVP.rds')
# Increase by 0.5Mb
parameter: increase_by = 500000

In [ ]:
# Convert RDS file to text file
[extract_1]
depends: R_library('data.table')
input: filename
output: f'{cwd:a}/{_input:bn}.gz'
R: expand = "${ }"
    dat = readRDS(${_input:r})
    # make rowname the first column
    # `check.names=TRUE` will 
    data.table::fwrite(data.frame(dat, check.names=TRUE, stringsAsFactors=FALSE), ${_output:r}, row.names=TRUE, sep=',')

In [ ]:
[extract_2]
depends: Py_Module('pandas', autoinstall=True), Py_Module('pandasql', autoinstall=True)
parameter: get_top = False
python: expand = "${ }"
    import pandas as pd
    from pandasql import sqldf
    # load data
    print("loading data ...")
    dat = pd.read_csv(${_input:r}, header=0)
    dat.columns = ['variant_id'] + dat.columns[1:].to_list()
    phenotypes = dat.columns.values[1:]
    # get proper chr and pos
    print("adding chr and pos ...")
    new_cols = dat["variant_id"].str.split(":", n = 1, expand = True) 
    dat['chr'] = new_cols[0].astype(int)
    dat['pos'] = new_cols[1].astype(int)
    # Get ranges of each chromosome
    print("preparing block table ...")
    blk = sqldf("SELECT dat.chr, min(dat.pos), max(dat.pos) FROM dat GROUP BY dat.chr")
    # Generate blocks table the same format as the table for LD blocks
    def get_blocks(chrom, start, end):
        res = []
        j = 1
        prev = start
        for curr in range(start, end, ${increase_by}):
            if start == curr:
                continue
            res.append([chrom, prev, curr, f'chr{chrom}_{j}'])
            prev = curr
            j += 1
        if curr < end:
            res.append([chrom, curr, end, f'chr{chrom}_{j}'])
        return res
    blk = sum([get_blocks(row[0], row[1], row[2]) for idx, row in blk.iterrows()], [])
    blk = pd.DataFrame(blk, columns = ['chr', 'start', 'end', 'blk_id'])
    blk.to_csv("${filename:n}" + '.blk.gz', index=False)
    # the core queries, first create a joint table with blk_id annotation, then query from there
    print("joining tables ...")
    query = f"SELECT dat.variant_id, dat.chr, dat.pos, {', '.join(['dat.' + y for y in phenotypes])}, blk.start, blk.end, blk.blk_id FROM dat LEFT JOIN blk WHERE dat.chr == blk.chr AND (dat.pos >= blk.start AND dat.pos <= blk.end)"
    dat = sqldf(query)   
    print("running variant extraction queries ...")
    if (${get_top}):
        queries = [f"SELECT variant_id, chr, pos, min({y}), start, end, blk_id FROM dat GROUP BY blk_id ORDER BY chr, pos" for y in phenotypes]
    else:
        queries = [f"SELECT variant_id, chr, pos, {y}, start, end, blk_id FROM dat ORDER BY chr, pos" for y in phenotypes]        
    res = [sqldf(query) for query in queries]
    # save result
    for idx, item in enumerate(res):
        item.drop(columns=['chr', 'pos'], inplace=True)
        item.columns = ['variant_id', phenotypes[idx], 'bin_start', 'bin_end', 'blk_id']
        item.to_csv("${cwd}/${filename:bn}" + f'.{phenotypes[idx]}.gz', sep='\t', index=False)

## Additional steps for input with `torus` pipeline

See the workflow [here](https://github.com/cumc/bioworkflows/blob/master/fine-mapping/gwas_enrichment.ipynb).

### LD blocks

In [ ]:
#fn=combined_pm_MVP
fn=original_z_mvp
zcat $fn.blk.gz | tail -n+2 | sed 's/^/chr/g' > $fn.blocks.txt

### SNP map and z-scores

In [ ]:
for i in hdl ldl tc tg; do
    zcat $fn.$i.gz | tail -n+2  | awk '{print $1,$5,$2}' OFS='\t' | gzip --best > $fn.$i.zscore.gz
    zcat $fn.$i.zscore.gz | cut -f1 | awk -F':' '{print "chr"$1"."$2,$1,$2}' OFS='\t' | gzip --best > $fn.$i.smap.gz
done